In [1]:
import pandas as pd
import numpy as np
import random
import scipy.stats as stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn import cross_validation

Steps:
1) Initial population (20) chromosomes:
    - 3 parts: [n-estimators][learning-rate][features]
               [600-1200]    [0.01-0.07]    [0,1 presence]
    - n-estimators: randint(0, 30)*20+600
    - learning-rate: randint(1, 5)*0.01
    - features: if random() > 0.7, include feature
2) Determine fitness:
    - feed values to gbr, get score
        - for n-estimators, round to nearest 10
    - fitness = wa * accuracy + wf * (1/sum(feature-presence))
        - wa, wf weights, wf meant to penalize incorporating large number of features
        - probably wa ~95-100 (num-features does not matter)
3) Hold out "elite" (top ~10%)
4) Crossover
    - repeat while len(newPop) < len(oldPop):
    - select of 2 parents: select random number r between 0 and sum of pop's fitness
                           sum through population until reach a number > r 
    - choose random crossover point, switch all after that
5) Mutation
    - for each bit, if random() < 0.001, flip


Overall:
Generate population
for x iterations:
    * evolve population (grade fitness, hold out, crossover, mutation -> return newPop)
return fittest

In [2]:
orig_data = pd.DataFrame.from_csv('Clean_data/train.csv', header=0, index_col=False)
df = pd.DataFrame.from_csv('Train_data/with_targets.csv')
df.columns

Index(['CELL_LINE', 'COMPOUND_A', 'COMPOUND_B', 'MAX_CONC_A', 'MAX_CONC_B',
       'IC50_A', 'H_A', 'Einf_A', 'IC50_B', 'H_B', 'Einf_B', 'A_HBA',
       'A_cLogP', 'A_HBD', 'A_Lipinski', 'A_MW', 'A_ALogP/XLogP', 'A_PSA',
       'A_#RotBonds', 'A_Arom Rings', 'A_Heavy atoms', 'A_QED', 'B_HBA',
       'B_cLogP', 'B_HBD', 'B_Lipinski', 'B_MW', 'B_ALogP/XLogP', 'B_PSA',
       'B_#RotBonds', 'B_Arom Rings', 'B_Heavy atoms', 'B_QED', 'GEX_CLUSTER',
       'ORIGIN_CLUSTER', 'THERAPY_CLUSTER', 'Target_share'],
      dtype='object')

In [3]:
df.columns[22]

'B_HBA'

#### feature chromosome represents: columns[11-21] -> [22-32]

In [4]:
num_features = 11 #(just the added features)
num_features

11

In [5]:
# returns binary string of sepcified length
def toBinary(x, length):
    bstring = bin(x)[2:]
    return (length-len(bstring))*'0'+bstring

def generatePop(count):
    pop = []
    for i in range(count):
        c1 = toBinary(random.randint(0, 30), 5)
        c2 = toBinary(random.randint(1, 7), 3)
        c3arr = [int(random.random()>0.5) for x in range(num_features)]
        c3 = ''.join(str(x) for x in c3arr)
        chromo = c1+c2+c3
        pop.append(chromo)
    return pop

In [6]:
print(generatePop(5))

['0110010100100001010', '0101001010000110110', '0111111011010111001', '1000001110000101101', '0100110001001110010']


In [7]:
def fitness(ind, wa):
    c1 = ind[:5]
    c2 = ind[5:8]
    c3 = ind[8:]
    num_est = int(c1, 2)*20+600
    lr = int(c2, 2)*0.01
    feats = [int(x) for x in list(c3)]
    
    # drop features whose corresponding position is 0:
    toDrop = []
    for i in range(len(feats)):
        if feats[i]==0:
            toDrop.append(df.columns[i+11]) # A_features
            toDrop.append(df.columns[i+22]) # B_features
    df_mod = df.drop(toDrop, axis=1)
    
    # build up X, y train/test values:
    df_mod = df_mod.iloc[np.random.permutation(len(orig_data))]
    Xpd = pd.get_dummies(df_mod, columns=['CELL_LINE', 'COMPOUND_A', 'COMPOUND_B'])
    Ypd = orig_data['SYNERGY_SCORE'].reindex(df_mod.index)
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(Xpd.values, Ypd.values)
    
    
    # train model
    gbr = GradientBoostingRegressor(n_estimators=num_est, learning_rate=lr, 
                                    max_depth=7, max_features='log2')
    gbr.fit(X_train, y_train)
    #accuracy = gbr.score(X_test, y_test)
    accuracy = stats.pearsonr(y_test, gbr.predict(X_test))[0]
    
    # calculate fitness score: wa (weight accuracy) vs wf (weight features = inverse size)
    fitness = wa*accuracy + (1-wa)*(1/sum(feats))
    return fitness

In [8]:
def select(pop):
    fitnesses = [fitness(ind, 0.9) for ind in pop]
    r = random.random()*sum(fitnesses)
    count = 0
    for i in range(len(pop)):
        count += fitnesses[i]
        if count > r:
            return pop[i]

def crossover(p1, p2):
    while True:
        crossSite = random.randint(1, len(p1)-2)
        c1 = p1[:crossSite] + p2[crossSite:]
        c2 = p2[:crossSite] + p1[crossSite:]
        if '1' in c1[5:8] and '1' in c2[5:8]: # if learning rate not 0
            break
    return c1, c2

def mutate(p1, muteRate):
    mutated = ''
    for bit in p1:
        if random.random() < muteRate:
            mutated += ('1' if (bit=='0') else '0')
        else:
            mutated += bit
    return mutated

In [9]:
def evolve(pop, muteRate=0.001):
    # "Hold-out" elite
    scores = [(fitness(ind, 0.95), ind) for ind in pop] # pairs of (fitness, individual)
    ranked = [x[1] for x in sorted(scores)] # ranked individuals by fitness
    retain_pos = int(0.1*len(ranked))
    newPop = ranked[:retain_pos]
    
    # Populate with children
    while (len(newPop) < len(pop)):
        p1, p2 = select(pop), select(pop)
        c1, c2 = crossover(p1, p2)
        c1, c2 = mutate(c1, muteRate), mutate(c2, muteRate)
        newPop.extend([c1, c2])
    return newPop

In [10]:
def bestFit(pop):
    scores = [(fitness(ind, 0.95), ind) for ind in pop] # pairs of (fitness, individual)
    best = sorted(scores)[0]
    return best[1], best[0] # individual, fitness

In [ ]:
p = generatePop(20)
print(bestFit(p)[1])
for i in range(15):
    p = evolve(p)
    print(bestFit(p)[1])